In [5]:
import cv2
import os
import numpy as np
import mediapipe as mp
from mediapipe.python.solutions.holistic import Holistic
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import TensorBoard

In [7]:
mp_holistic = mp.solutions.holistic  # Holistic model
mp_drawing = mp.solutions.drawing_utils  # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    image.flags.writeable = False
    results = model.process(image)  # Make prediction
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert RGB back to BGR
    return image, results

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    return np.concatenate([pose, face, lh, rh])

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [9]:
DATA_PATH = os.path.join('DATA SET 2')
actions = np.array(['hello', 'Indian', 'bye', 'my', 'you', 'man', 'girl', 'name', 'thank you', 'what', 'yes', 'no'])
sequence_length = 30

In [11]:
label_map = {label: num for num, label in enumerate(actions)}
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(30, 1662)))
model.add(MaxPooling1D(2))
model.add(Conv1D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [15]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
model.fit(X_train, y_train, epochs=500, callbacks=[tb_callback])

Epoch 1/500
9/9 [==============================] - 2s 16ms/step - loss: 2.4893 - categorical_accuracy: 0.0625
Epoch 2/500
9/9 [==============================] - 0s 15ms/step - loss: 2.4840 - categorical_accuracy: 0.0694
Epoch 3/500
9/9 [==============================] - 0s 15ms/step - loss: 2.4835 - categorical_accuracy: 0.0833
Epoch 4/500
9/9 [==============================] - 0s 15ms/step - loss: 2.4769 - categorical_accuracy: 0.0972
Epoch 5/500
9/9 [==============================] - 0s 15ms/step - loss: 2.4650 - categorical_accuracy: 0.0799
Epoch 6/500
9/9 [==============================] - 0s 17ms/step - loss: 2.4111 - categorical_accuracy: 0.1424
Epoch 7/500
9/9 [==============================] - 0s 15ms/step - loss: 2.3920 - categorical_accuracy: 0.1736
Epoch 8/500
9/9 [==============================] - 0s 16ms/step - loss: 2.4323 - categorical_accuracy: 0.1701
Epoch 9/500
9/9 [==============================] - 0s 16ms/step - loss: 2.3401 - categorical_accuracy: 0.1875
Epoch 10/5

KeyboardInterrupt: 

In [17]:
model.evaluate(X_test, y_test)

3/3 [==============================] - 0s 4ms/step - loss: 6.2623 - categorical_accuracy: 0.6667


[6.262277126312256, 0.6666666865348816]

In [19]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [21]:
yhat = model.predict(X_test)

3/3 [==============================] - 0s 5ms/step


In [23]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [25]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[63,  3],
        [ 0,  6]],

       [[61,  2],
        [ 0,  9]],

       [[64,  0],
        [ 3,  5]],

       [[64,  1],
        [ 2,  5]],

       [[60,  6],
        [ 3,  3]],

       [[70,  1],
        [ 1,  0]],

       [[65,  1],
        [ 1,  5]],

       [[64,  1],
        [ 0,  7]],

       [[69,  1],
        [ 1,  1]],

       [[65,  1],
        [ 3,  3]],

       [[63,  4],
        [ 2,  3]],

       [[60,  3],
        [ 8,  1]]], dtype=int64)

In [31]:
model.save('test 12-30.h5')

In [33]:
from tensorflow.keras.models import load_model

model = load_model('test 12-30.h5')

In [35]:
model.load_weights('test 12-30.h5')

In [37]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()


3/3 [==============================] - 0s 4ms/step


In [39]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[63,  3],
        [ 0,  6]],

       [[61,  2],
        [ 0,  9]],

       [[64,  0],
        [ 3,  5]],

       [[64,  1],
        [ 2,  5]],

       [[60,  6],
        [ 3,  3]],

       [[70,  1],
        [ 1,  0]],

       [[65,  1],
        [ 1,  5]],

       [[64,  1],
        [ 0,  7]],

       [[69,  1],
        [ 1,  1]],

       [[65,  1],
        [ 3,  3]],

       [[63,  4],
        [ 2,  3]],

       [[60,  3],
        [ 8,  1]]], dtype=int64)

In [41]:
accuracy_score(ytrue, yhat)

0.6666666666666666

In [43]:
actions = np.array(['hello', 'Indian', 'bye', 'my', 'you', 'man', 'girl', 'name', 'thank you', 'what', 'yes', 'no'])